## 8.1 EM algorithm for binary matrix completion

### (a) Sanity check

In [34]:
import math
import numpy as np

def read_file(filename):
    f = open(filename, 'r')
    data = []
    for line in f.readlines():
        line = line.strip('\n').split()
        if len(line) == 1:
            data.append(line[0])
        else:
            data.append(line)

    return data

rating = read_file('hw8_ratings_fa18.txt')
title = read_file('hw8_movieTitles_fa18.txt')
pid = read_file('hw8_studentPIDs_fa18.txt')
T = len(rating)

In [35]:
movie_popularity = {}
for i in range(len(title)):
    movie_popularity[title[i]] = 0
    amount = 0
    for t in range(T):
        if rating[t][i] != '?':
            amount += 1
        if rating[t][i] == '1':
            movie_popularity[title[i]] += 1
    movie_popularity[title[i]] /= float(amount)

movie_popularity = sorted(movie_popularity.items(), key=lambda item: item[1])
print(len(movie_popularity))


62


In [36]:
for k in movie_popularity:
    print(k)

('The_Last_Airbender', 0.3026315789473684)
('Fifty_Shades_of_Grey', 0.3333333333333333)
('I_Feel_Pretty', 0.36363636363636365)
('Chappaquidick', 0.45454545454545453)
('Man_of_Steel', 0.4892086330935252)
('Prometheus', 0.51)
('The_Shape_of_Water', 0.5111111111111111)
('Phantom_Thread', 0.5454545454545454)
('Magic_Mike', 0.5531914893617021)
('World_War_Z', 0.5784313725490197)
('Bridemaids', 0.6271186440677966)
('American_Hustle', 0.6486486486486487)
('Drive', 0.6666666666666666)
('The_Hunger_Games', 0.6683168316831684)
('Thor', 0.6934673366834171)
('Pitch_Perfect', 0.6962025316455697)
('Fast_Five', 0.704)
('Avengers:_Age_of_Ultron', 0.7208121827411168)
('Jurassic_World', 0.7216494845360825)
('The_Hateful_Eight', 0.7288135593220338)
('The_Revenant', 0.73)
('Dunkirk', 0.732824427480916)
('Star_Wars:_The_Force_Awakens', 0.7440476190476191)
('Mad_Max:_Fury_Road', 0.7443609022556391)
('Captain_America:_The_First_Avenger', 0.7524271844660194)
('The_Perks_of_Being_a_Wallflower', 0.7594936708860

### (e) Implementation

In [37]:
def read_file_EM(filename):
    f = open(filename, 'r')
    data = []
    for line in f.readlines():
        line = line.strip('\n').split()
        line = list(map(float, line))
        if len(line) == 1:
            data.append(line[0])
        else:
            data.append(line)

    return data

init_z = read_file_EM('hw8_probZ_init.txt')
init_z_given_r = read_file_EM('hw8_probRgivenZ_init.txt')

In [38]:
p = init_z
conditional_p = init_z_given_r

In [39]:
def compute_likelihood(p, conditional_p, rating_t):
    l = 0
    ind = 0
    part = []
    for i in range(len(p)):
        cur = p[i]
        for j in range(len(rating_t)):
            if rating_t[j] == '1':
                cur *= conditional_p[j][i]
            if rating_t[j] == '0':
                cur *= (1 - conditional_p[j][i])
        l += cur
        part.append(cur)
    return (l, part)

In [40]:
def compute_log_likelihood(p, conditional_p, rating):
    res = 0
    for t in range(T):
        res += math.log(compute_likelihood(p, conditional_p, rating[t])[0])
    return res/float(T)

In [41]:
def compute_posterior(p, conditional_p, rating):
    post = np.zeros((T, len(p)))
    for i in range(len(p)):
        for t in range(T):
            total, part = compute_likelihood(p, conditional_p, rating[t])
            post[t][i] = part[i] / total
    return post

In [42]:
def EM(p, conditional_p, rating):
    #E-step
    post = compute_posterior(p, conditional_p, rating)
    for i in range(len(p)):        #4
        total = sum(post[:, i])
        p[i] = total / T      #P(Z=i)
        for j in range(len(conditional_p)):       #P(Rj=1|Z=i)
            conditional_p_cur = []
            for r in rating[:, j]:
                if r == '?':
                    conditional_p_cur.append(conditional_p[j][i])
                else:
                    conditional_p_cur.append(int(r))
            temp = 0
            for t in range(T):
                temp += post[:, i][t] * conditional_p_cur[t]
            conditional_p[j][i] = temp / total
    return p, conditional_p

In [43]:
Iter = 129
k = 0
rating = np.array(rating)
t = 0
print('Iter 0' + ': %.4f' % compute_log_likelihood(p, conditional_p, rating))
while k < Iter:
    if k == pow(2, t):
        print('Iter ' + str(k) + ': %.4f' % compute_log_likelihood(p, conditional_p, rating))
        t += 1

    p, conditional_p = EM(p, conditional_p, rating)
    k += 1

Iter 0: -26.6788
Iter 1: -16.0947
Iter 2: -14.2878
Iter 4: -13.2651
Iter 8: -12.8473
Iter 16: -12.7060
Iter 32: -12.6407
Iter 64: -12.6161
Iter 128: -12.5912


### (f) Personal movie recommendations

In [44]:
print(pid.index('A53278437'))

140


In [45]:
post = compute_posterior(p, conditional_p, rating)

In [46]:
my_post = post[140]
print('My posterior probability is : ')
print('===============================')
for k in my_post:
    print(k)

My posterior probability is : 
0.112487757229
7.19181680071e-63
0.887482870153
2.93726181826e-05


In [49]:
id = 140
guess_list = {}
for i in range(len(rating[0])):
    if rating[id][i] == '?':
        guess = 0
        for t in range(4):
            total, part = compute_likelihood(p, conditional_p, rating[id])
            guess += conditional_p[i][t] * part[t] / total
        guess_list[title[i]] = guess

guess_list = sorted(guess_list.items(), key=lambda item:item[1], reverse=True)
print("(unseen) movie sorted by expected ratings: ")
print('=======================================')
for k in guess_list:
    print(k)

(unseen) movie sorted by expected ratings: 
('Hidden_Figures', 0.98106297393024366)
('Midnight_in_Paris', 0.96898420671970298)
('The_Hateful_Eight', 0.9636647408126191)
('Wolf_of_Wall_Street', 0.92367136065960742)
('Django_Unchained', 0.89482209687301839)
('Manchester_by_the_Sea', 0.87393689525203988)
('I_Feel_Pretty', 0.87294624767607853)
('The_Girls_with_the_Dragon_Tattoo', 0.86461457323121682)
('Frozen', 0.85361705787192577)
('Pitch_Perfect', 0.84718167242176612)
('Black_Swan', 0.83387687108383601)
('X-Men:_First_Class', 0.80067950653763575)
('Ready_Player_One', 0.78686091289682536)
('Toy_Story_3', 0.77199388082973452)
('21_Jump_Street', 0.76875253611076844)
('Ex_Machina', 0.73246341612779575)
('The_Help', 0.72112366160076291)
('Magic_Mike', 0.71795092142735872)
('The_Perks_of_Being_a_Wallflower', 0.65377275516949318)
('Mad_Max:_Fury_Road', 0.63459649739563584)
('Bridemaids', 0.61775037815550304)
('Phantom_Thread', 0.58696344096576969)
('The_Last_Airbender', 0.48903330340627393)
('H